# Convert Downloaded Data Files into CSVs

## Imports

In [1]:
import pandas as pd
import numpy as np
import csv
from pathlib import Path
import re

# Country Codes

In [30]:
def convert_country_codes(file_path, csv_target_path):
    print('Converting country codes to .csv file...')
    file = open(file_path, 'r')
    lines = file.readlines()

    header = ['Code', 'Country']
    to_csv_lines = []

    for line in lines:
        modified_line_pieces = []
        # code is always the first 3 digits
        code = line[:3]

        # country always starts at index 4
        country = (line[4:]).rstrip()

        modified_line_pieces.append(str(code))
        modified_line_pieces.append(country)
        to_csv_lines.append(modified_line_pieces)
    
    with open(csv_target_path, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(header)
        writer.writerows(to_csv_lines)
    
    print('Done.')

In [31]:
path = '../../og_data_files/v2.country.codes.txt'
target_path = '../country_codes.csv'     # include path to file and expected file name here

convert_country_codes(path, target_path)

Converting country codes to .csv file...
Done.


# Station Metadata

### Convert text to csv

Converts what's in the text file to csv format. Nothing else is changed.

In [54]:
def convert_station_metadata_v1(file_path, csv_target_path):
    print('Converting station metadata text file to csv...')

    file = open(file_path, 'r')
    lines = file.readlines()

    header = ['Station', 'Location', 'Latitude', 'Longitude', 'Elevation']       #ignore country for now, find it later

    # line_pattern = r"([0-9]{11})\s+([A-Z./\-\(\)%<>&?',\"\s]*)\s+(-*[0-9]{1,2}\.[0-9]{2})\s+(-*[0-9]{1,3}\.[0-9]{2})\s+(-*[0-9]{1,5})"
    line_pattern = r"([0-9]{11})\s+(.*)\s+(-*[0-9]{1,2}\.[0-9]{2})\s+(-*[0-9]{1,3}\.[0-9]{2})\s+(-*[0-9]{1,5})"
    location_country_pattern = r"(^\s*\s?^\s*)\s+"      #count all white space positions

    to_csv_lines = []

    problem_count = 0
    problem_lines = []

    for line in lines:
        # print(line)
        modified_line_pieces = []

        capture = re.findall(line_pattern, line)[0]

        location_country = (capture[1]).rstrip()        #strip trailing whitespaces
        print(location_country)

        # whitespaces = [i for i, ltr in enumerate(location_country) if ltr == ' ']       # find all whitespaces
        whitespaces = [(p.start(), p.end()) for p in re.finditer('\s{2,}', location_country)]
        print(whitespaces)

        if len(location_country) == 0:          # no location or country
            location = ''
        elif len(whitespaces) == 0:             # format 'location'
            location = location_country
        elif len(whitespaces) == 1:
            location = location_country[:whitespaces[0][0]]     # format 'location      country'
        elif len(whitespaces) > 1:              # format 'loca  tion        country'
            location = ''
            problem_count += 1
            problem_lines.append(location_country)
        else:
            location = ''


        # if(len(location_country) == 0):         # no location or country
        #     location = ''
        # elif len(whitespaces) == 0:             # format 'location'
        #     location = location_country
        # elif len(whitespaces) == 1 and (whitespaces[0][1] - whitespaces[0][0] == 1):    # format 'lo ca ti on'
        #     location = location_country
        # else:
        #     for w in whitespaces:           # if the space is greater than 1, this denotes a tab and where location and country split
        #         if w[1] - w[0] > 1 and w[0] != 0:       # format 'location      country'
        #             location = location_country[:w[1]]
        #         elif w[1] - w[0] > 1 and w[0] == 0:     # format '          country'
        #             location = ''
        #         else:
        #             location = ''
        
        # print(location)

        # ughhhh the format for some lines is so messed up this isn't working

        print('---')

        # if len(location_country) > 0:
        #     location = re.match(r"\s", location_country)
        #     print(location)
        # else:
        #     location = ''

        # modified_line_pieces.append(capture[0].strip())

    # with open(csv_target_path, 'w') as csv_file:
    #     writer = csv.writer(csv_file)
    #     writer.writerow(header)
    #     writer.writerows(to_csv_lines)
    
    print(problem_count)
    print(problem_lines)
    print('Done.')

In [68]:
def convert_station_metadata(file_path, csv_target_path):
    print('Converting station metadata text file to csv...')

    file = open(file_path, 'r')
    lines = file.readlines()

    header = ['Station', 'Location', 'Latitude', 'Longitude', 'Elevation']       #ignore country for now, find it later

    line_pattern = r"([0-9]{11})\s+(.*)\s+(-*[0-9]{1,2}\.[0-9]{2})\s+(-*[0-9]{1,3}\.[0-9]{2})\s+(-*[0-9]{1,5})"
    location_country_pattern = r"(^\s*\s?^\s*)\s+"      #count all white space positions

    to_csv_lines = []

    problem_count = 0
    problem_lines = []

    for line in lines:
        modified_line_pieces = []

        capture = re.findall(line_pattern, line)[0]
        location_country = (capture[1])
        print(location_country)
        
        modified_line_pieces.append(capture[0])
        modified_line_pieces.append(location_country)
        modified_line_pieces.append(capture[2])
        modified_line_pieces.append(capture[3])
        modified_line_pieces.append(capture[4])

        to_csv_lines.append(modified_line_pieces)
        
        # countries start at index 20   - exclude from station name
        # regions start at index 16     - include in the station name and format fix
        # no this is too inconsistent
        # pick out the ones with special cases that need handling - general cases for the rest
        # assumes that regular case is country starts at index 20 (can cut before)
        # station codes have .1 or .2 (extra decimal place number) if last numbers are 001 or 002 (respective) instead of 000
        # some like row TSHOLOTSHO are too long and cross over until after index 20 - figure out some way to deal with this

        # weird country list: 
        # 161, 
        # 162 (when there's (R. S. A.) there too), 
        # 168, 
        # 202, 
        # 207 (india ones include region names too sometimes, not part of station name though)
        # 210 some okinawa too long, MARCUS IS/MINAMITORIPACIFIC OC is weird
        # 219 (some pakistan includes region names, not part of station name)
        # 221 UIJONGBU/CAMP RED CLOUD weird
        # 229 some have ??? but not part of station name
        # 230
        # 303 some brazil have 2 letter abbrvs beginning at index 22 - what do they represent? not part of station name, some names too long
        # 403 canada has no country - ignore index cutting
        # 416 some panama weird
        # 424
        # 425 lots of US names too long

    with open(csv_target_path, 'w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(header)
        writer.writerows(to_csv_lines)

    print('Done.')

In [69]:
path = '../../og_data_files/v2.prcp.inv.txt'
target_path = '../station_metadata.csv'     # include path to file and expected file name here

convert_station_metadata(path, target_path)

Converting station metadata text file to csv...
ALGIERS             ALGERIA    
SKIKDA              ALGERIA    
ANNABA              ALGERIA    
ALGER/DAR EL BEIDA  ALGERIA    
FT.NATIONAL                    
CAP CARBON          ALGERIA    
BEJAIA              ALGERIA    
DJIDJELI            ALGERIA    
GUELMA                         
CONSTANTINE         ALGERIA    
CHLEF               ALGERIA    
ORLEANS VILLE                  
MILIANA             ALGERIA    
BORDJ BOU ARRERIDJ  ALGERIA    
SETIF                          
MOSTAGANEM          ALGERIA    
BATNA               ALGERIA    
TEBESSA                        
ORAN/ES SENIA       ALGERIA    
MASCARA                        
BISKRA              ALGERIA    
TLEMCEN/ZANATA      ALGERIA    
DJELFA              ALGERIA    
LAGHOUAT            ALGERIA    
MECHERIA            ALGERIA    
SIDI-BEL-ABBAS                 
GERYVILLE                      
TOUGGOURT           ALGERIA    
EL ALIOD            ALGERIA    
EL OUED             ALGE

In [83]:
# fix metadata

metadata = pd.read_csv('../station_metadata.csv')
number_array = metadata['Station'].tolist()
location_array = metadata['Location'].tolist()
# print(location_array)

location = []
region = []
country = []

for i in range(len(location_array)):
    print(location_array[i])
    if len(str(location_array[i]).strip()) == 0:
        location.append('')
        region.append('')
        country.append('')
    elif str(number_array[i])[:3] == '207' and (location_array[i])[20:16] != 'INDIA':
        location.append(str(location_array[i])[:16].rstrip())
        region.append(str(location_array[i])[16:].rstrip())
        country.append('')
    elif str(number_array[i])[:3] == '219' and (location_array[i])[20:29] != 'PAKISTAN':
        location.append(str(location_array[i])[:16].rstrip())
        region.append(str(location_array[i])[16:].rstrip())
        country.append('')
    else:
        location.append(str(location_array[i])[:20].rstrip())
        region.append('')
        country.append(str(location_array[i])[20:].rstrip())

ALGIERS             ALGERIA    
SKIKDA              ALGERIA    
ANNABA              ALGERIA    
ALGER/DAR EL BEIDA  ALGERIA    
FT.NATIONAL                    
CAP CARBON          ALGERIA    
BEJAIA              ALGERIA    
DJIDJELI            ALGERIA    
GUELMA                         
CONSTANTINE         ALGERIA    
CHLEF               ALGERIA    
ORLEANS VILLE                  
MILIANA             ALGERIA    
BORDJ BOU ARRERIDJ  ALGERIA    
SETIF                          
MOSTAGANEM          ALGERIA    
BATNA               ALGERIA    
TEBESSA                        
ORAN/ES SENIA       ALGERIA    
MASCARA                        
BISKRA              ALGERIA    
TLEMCEN/ZANATA      ALGERIA    
DJELFA              ALGERIA    
LAGHOUAT            ALGERIA    
MECHERIA            ALGERIA    
SIDI-BEL-ABBAS                 
GERYVILLE                      
TOUGGOURT           ALGERIA    
EL ALIOD            ALGERIA    
EL OUED             ALGERIA    
EL ARFIANE          ALGERIA    
AIN SEFR

In [84]:
metadata = metadata.drop(['Location'], axis=1)
metadata.insert(1, 'Country', country)
metadata.insert(1, 'Region', region)
metadata.insert(1, 'Location', location)

In [85]:
metadata.to_csv('../../new_data_files/station_metadata_fix_1.csv', index=False)

# fix the rest by hand - most of it is correct
# dude the consistency of this dataset format is terrible

In [89]:
mm = pd.read_csv('../station_metadata_fixed.csv')

print(mm)
print(mm.shape)

           Station            Location Region     Country  Latitude  \
0      10160354001             ALGIERS    NaN     ALGERIA     36.70   
1      10160355000              SKIKDA    NaN     ALGERIA     36.90   
2      10160360000              ANNABA    NaN     ALGERIA     36.80   
3      10160390000  ALGER/DAR EL BEIDA    NaN     ALGERIA     36.70   
4      10160395002         FT.NATIONAL    NaN         NaN     36.52   
...            ...                 ...    ...         ...       ...   
20585  70089592000               MIRNY    NaN  ANTARCTICA    -66.60   
20586  70089606000              VOSTOK    NaN  ANTARCTICA    -78.50   
20587  70089611000   WILKES STA./CASEY    NaN  ANTARCTICA    -66.30   
20588  70089664000       MCMURDO SOUND    NaN         NaN    -77.88   
20589  80091155002              SHIP P    NaN     OCEANIA     49.00   

       Longitude  Elevation  
0           3.60        290  
1           7.00          9  
2           7.80          4  
3           3.30         25

## Detailed Station Metadata

In [69]:
# assumes the format from the fixed metadata conversion
def station_metadata_add_detail(country_path, station_path, csv_target_path):
    print('Converting station metadata text file to csv...')

    country_df = pd.read_csv(country_path)
    station_df = pd.read_csv(station_path)

    country_dict = dict(zip(country_df['Code'].astype("string"), country_df['Country']))     # convert to dictionary to eliminate need for searching
    # print(country_dict)

    header = ['Station', 'Identifier', 'Name', 'Region', 'Country', 'Latitude', 'Longitude', 'Elevation']

    to_csv_lines = []

    for row in range(station_df.shape[0]):
        modified_line_pieces = []
        identifier = str(station_df.iat[row, 0])        # index 0 is station
        country_code = identifier[:3]
        station_code = identifier[3:8]
        duplicate_code = (identifier[9:]).lstrip('0')       # strip leading 0s
        station_code = f'{station_code}.{duplicate_code}' if len(duplicate_code) > 0 else station_code

        modified_line_pieces.append(station_code)
        modified_line_pieces.append(identifier)
        modified_line_pieces.append(station_df.iat[row, 1])         # index 1: location
        modified_line_pieces.append('' if str(station_df.iat[row, 2]) == 'nan' else station_df.iat[row, 2])         # index 2: region
        modified_line_pieces.append(country_dict[country_code])
        modified_line_pieces.append(station_df.iat[row, 4])         # index 4: latitude
        modified_line_pieces.append(station_df.iat[row, 5])         # index 5: longitude
        modified_line_pieces.append(station_df.iat[row, 6])         # index 6: elevation

        to_csv_lines.append(modified_line_pieces)

    with open(csv_target_path, 'w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(header)
        writer.writerows(to_csv_lines)

    print('Done.')

In [70]:
country_path = '../country_codes.csv'
station_path = '../station_metadata_fixed.csv'
detailed_station_path = '../detailed_station_metadata.csv'

station_metadata_add_detail(country_path, station_path, detailed_station_path)

Converting station metadata text file to csv...
Done.


## Update Metadata Regions

In [ ]:
# ehh only canada and india consistently have regions defined - do this by hand
# double check the regions for brazil -- wtf are they??? can't find any info on "FZ" or some of the others mentioned

# Station Precipitation Data

## Convet text to csv

In [3]:
def convert_station_prcp_data(file_path, csv_target_path):
    print('Converting station precipitation data text file to csv...')

    file = open(file_path, 'r')
    lines = file.readlines()

    header = ['Station', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

    to_csv_lines = []

    for line in lines:
        modified_line_pieces = []
        # split_line = line.split()      # works well until you get to -9999, then they're stuck together
        # for piece in split_line:
        #     if '-9999' in piece:
        #         nulls = piece.split('-')
        #         modified_line_pieces += ((null if (null != '9999' and null != '8888') else ('-9999' if null == '9999' else '-8888')) for null in nulls)
        #     if '-8888' in piece:
        #         bads = piece.split('-')
        #         modified_line_pieces += ((bad if bad != '8888' else '-8888') for bad in bads)
        #     else:
        #         modified_line_pieces.append(piece)

        # split by indexing instead (more consistent)
        modified_line_pieces.append(line[:16])          # identifier takes up the first characters
        readings = re.findall('.{5}', line[16:])        # every reading takes up 5 characters
        modified_line_pieces.extend([reading.lstrip() for reading in readings])

        # print(modified_line_pieces)
        to_csv_lines.append(modified_line_pieces)

    with open(csv_target_path, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(header)
        writer.writerows(to_csv_lines)
    
    print('Done.')

In [4]:
path = '../../og_data_files/v2.prcp.txt'
target_path = '../prcp.csv'     # include path to file and expected file name here

convert_station_prcp_data(path, target_path)

Converting station precipitation data text file to csv...
Done.


## Detailed Precipitation Data

In [1]:
# assumes the format from the fixed metadata conversion
def prcp_add_detail_v1(prcp_path, station_path, csv_target_path):
    print('Converting station metadata text file to csv...')

    prcp_df = pd.read_csv(prcp_path)
    station_df = pd.read_csv(station_path)

    header = ['Station', 'Identifier', 'Year']
    readings_header = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

    # np array for monthly readings to make it take less time
    readings_array = ((prcp_df.iloc[:, 1:]).astype('float')).to_numpy()
    readings_array = readings_array/10      # get millimeters
    readings_df = pd.DataFrame(readings_array, columns=readings_header)
    # print(readings_array)

    station_list = []
    identifier_list = []
    year_list = []

    for row in range(prcp_df.shape[0]):
        identifier = str(prcp_df.iat[row, 0])
        station_code = identifier[3:8]
        duplicate_code = (identifier[8:11]).lstrip('0')       # strip leading 0s
        station_code = f'{station_code}.{duplicate_code}' if len(duplicate_code) > 0 else station_code
        year = identifier[-4:]

        station_list.append(station_code)
        identifier_list.append(identifier_list)
        year_list.append(year)
    
    readings_df.insert(0, 'Year', year_list)
    readings_df.insert(0, 'Identifier', identifier_list)
    readings_df.insert(0, 'Station', station_list)

    print(f'shape is: {readings_df.shape}')

    # readings_df.to_csv(csv_target_path, index=False)

    print('Done.')

In [5]:
# assumes the format from the fixed metadata conversion
def prcp_add_detail(prcp_path, csv_target_path):
  print('Converting station metadata text file to csv...')

  prcp_df = pd.read_csv(prcp_path)

  # header = ['Station', 'Identifier', 'Year']
  readings_header = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

  header = ['Station', 'Identifier', 'Year', 'January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

  # np array for monthly readings to make it take less time
  readings_array = ((prcp_df.iloc[:, 1:]).astype('float')).to_numpy()
  readings_array = readings_array/10      # get millimeters
  readings_df = pd.DataFrame(readings_array, columns=readings_header)
  # print(readings_array)

  to_csv_lines = []

  # station_list = []
  # identifier_list = []
  # year_list = []

  # print(readings_df.shape)

  # for row in range(prcp_df.shape[0]):
  #     identifier = str(prcp_df.iat[row, 0])
  #     station_code = identifier[3:8]
  #     duplicate_code = (identifier[9:12]).lstrip('0')       # strip leading 0s
  #     station_code = f'{station_code}.{duplicate_code}' if len(duplicate_code) > 0 else station_code
  #     year = identifier[-4:]

  #     station_list.append(station_code)
  #     identifier_list.append(identifier_list)
  #     year_list.append(year)

  #     print(f'line {row}/{prcp_df.shape[0]}')
  #     # print(f'code: {station_code}, id: {identifier}, year: {year}')
  
  # readings_df.insert(0, 'Year', year_list)
  # readings_df.insert(0, 'Identifier', identifier_list)
  # readings_df.insert(0, 'Station', station_list)

  # print(f'shape is: {readings_df.shape}')

  # readings_df.to_csv(csv_target_path, index=False)      # holy shit why is this taking up so much space

  # gonna try csv writer instead. hopefully its less awfu
  
  for row in range(prcp_df.shape[0]):
    modified_line_pieces = []
    identifier = str(prcp_df.iat[row, 0])
    station_code = identifier[3:8]
    duplicate_code = (identifier[8:11]).lstrip('0')       # strip leading 0s
    station_code = f'{station_code}.{duplicate_code}' if len(duplicate_code) > 0 else station_code
    year = identifier[-4:]

    modified_line_pieces.append(station_code)
    modified_line_pieces.append(identifier)
    modified_line_pieces.append(year)

    for month in range(0, 12):
      modified_line_pieces.append(readings_df.iat[row, month])
    
    to_csv_lines.append(modified_line_pieces)
    
    print(f'line {row}/{prcp_df.shape[0]}')
  
  with open(csv_target_path, 'w') as csv_file:
      writer = csv.writer(csv_file)
      writer.writerow(header)
      writer.writerows(to_csv_lines)
  
  print('Done.')

In [6]:
prcp_path = '../prcp.csv'
detailed_prcp_path = '../detailed_prcp_data.csv'

prcp_add_detail(prcp_path, detailed_prcp_path)

Converting station metadata text file to csv...
